In [1]:
from algoneer_datasets.adult import load_dataset
ds = load_dataset()
df = ds.df

In [2]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
X = df.drop('income', axis=1)
Y = df['income']

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.25, stratify=Y, random_state=10,
)

In [4]:

#Support Vector Machine Based Classifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from collections import defaultdict

scores = defaultdict(dict)

classifiers = {
    'logistic-regression' : LogisticRegression,
    'k-nearest-neighbors' : KNeighborsClassifier,
    'decision-tree' : DecisionTreeClassifier,
    'gaussian-nb' : GaussianNB
    
}

In [5]:
clf = LogisticRegression()

In [6]:
clf.fit(X_train, Y_train)

/home/andreas/projects/geordi/algoneer/algoneer/venv/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: could not convert string to float: ' United-States'

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

Y_pred_test = clf.predict(X_test) == Y_test

print(accuracy_score(Y_pred_test, Y_test))

## Partial Dependence Plot

To make a partial dependence plot for a given feature, we implement the following algorithm:

* If the feature is categorical, we generate a list of all possible feature values.
* If the features is numerical, we generate a list of values spanning the valid range of the feature
* For each value from the list of values, we calculate the average prediction value (for regression) or the class frequencies (for classification) over all datapoints in the dataset.
* We plot the resulting value as a function of the feature value.

In [ ]:
def pdp_categorical(X, model, column):
    """
    Generate the partial dependence
    """
    values = X[column]
    uniques = values.unique()
    YS = []
    for v in uniques:
        NX = X.copy()
        NX[column] = v
        Y = clf.predict(NX)
        YS.append(sum(Y)/len(Y))
    return uniques, sorted(YS)

In [ ]:
x, pdv = pdp_categorical(X_test, clf, "hours-per-week")

In [ ]:
import matplotlib.pylab as pl

In [ ]:
pl.plot(pdv)